In [2]:
import Pkg; 

if split(pwd(),"/")[end] == "prototyping"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2
using PyCall

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [4]:
function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

function get_polar_selection(points, n_atoms_per_mol::Int, negatively_charged::Vector{Int}, positively_charged::Vector{Int})
    n_mol = Int(length(points) ÷ n_atoms_per_mol)
    negatively_charged = vcat([negatively_charged .+ i*n_atoms_per_mol for i in 0:n_mol-1]...)
    positively_charged = vcat([positively_charged .+ i*n_atoms_per_mol for i in 0:n_mol-1]...)
    [points[negatively_charged]; points[positively_charged]], [[0 for _ in 1:length(negatively_charged)]; [1 for _ in 1:length(positively_charged)]]
end

get_polar_selection (generic function with 1 method)

In [ ]:
function state_to_poly(flat_realization::Vector, radii::Vector, filepath::String, n_mol::Int, n_atoms_per_mol::Int)
    open(string(filepath, ".poly"), "w") do io
        println(io,"POINTS")
        color = ""
        for i in 0:n_mol-1
            color = "c($(rand()),$(rand()),$(rand())"
            for j in 0:n_atoms_per_mol-1
                atom_id = (i * n_atoms_per_mol + j)
                x = atom_id*3 + 1
                y = atom_id*3 + 2
                z = atom_id*3 + 3
                println(io, "$(i * n_atoms_per_mol + j + 1): $(flat_realization[x]) $(flat_realization[y]) $(flat_realization[z]) $(radii[atom_id+1]) $(color), $(radii[atom_id+1]))")
            end
        end
        println(io,"POLYS")
        println(io,"END")
    end
end

In [5]:
template_centers = MorphoMol.Utilities.TMV_TEMPLATES["6r7m"]["template_centers"]

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [12]:
template_centers[:,[3,4,5,6,7,8]]

3×6 Matrix{Float64}:
 -26.5597   -27.5404   -26.6276   -27.6097   -25.358    -25.1549
   9.03875    8.57911   11.0769    11.6147     9.11043    8.90469
  -2.71995   -2.14188   -4.15425   -3.28763   -2.1614    -0.736389